Para este desafio, temos um dataset contendo dados sobre a reformulação de um site de vendas, com amostras coletadas antes e após as atualizações.

Um grupo de controle - Eles viram o design antigo

Um grupo de tratamento (ou experimental) - Eles viram o novo design

0 - O usuário não comprou o produto durante esta sessão de usuário

1 - O usuário comprou o produto durante esta sessão de usuário

Vamos formular nossas hipóteses:

## Hₒ: p = pₒ - as proporções populacionais são iguais.

## Hₐ: p ≠ pₒ - as proporções populacioneis são diferentes

## Também definiremos um nível de confiança de 95%

# Carregando as bibliotecas necessárias

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as st
import statsmodels.stats.api as sms
from math import ceil

# Carregando os dados do dataframe

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/EBA/Datasets/df_test.csv')
df.head()

,Unnamed: 0,user_id,timestamp,landing_page,converted,group
0,281655,918193,2017-01-03 13:42:27.285433,new_page,1,treatment
1,7271,679658,2017-01-20 02:00:54.448676,new_page,1,treatment
2,269760,815158,2017-01-16 05:11:33.823264,new_page,1,treatment
3,158882,857307,2017-01-22 08:58:50.687626,new_page,1,treatment
4,153958,716896,2017-01-15 17:59:03.065352,new_page,1,treatment


In [ ]:
df.columns

Index(['Unnamed: 0', 'user_id', 'timestamp', 'landing_page', 'converted',
       'group'],
      dtype='object')

# Criando dois conjuntos de dados para separar os grupos (Controle versus Tratamento)

In [ ]:
grupo_tratamento = df[df.group == 'treatment'] # grupo tratamento
grupo_controle = df[df.group == 'control'] #grupo controle

In [ ]:
# calculando a taxa de conversão para cada grupo (conversão = n° de pessoas que compraram o produto vs total de pessoas que entraram no site)

conversao_tratamento = grupo_tratamento[grupo_tratamento.converted == 1].user_id.count()/grupo_tratamento.user_id.count()

conversao_controle = grupo_controle[grupo_controle.converted == 1].user_id.count()/grupo_controle.user_id.count()

print("A taxa de conversão do grupo tratamento é:", conversao_tratamento)
print("A taxa de conversão do grupo controle é:", conversao_controle)

A taxa de conversão do grupo tratamento é: 0.18985095812633074
A taxa de conversão do grupo controle é: 0.12455642299503193


Aparentemente, houve uma melhora na média entre os dois grupos (Controle 12% e Tratamento 18%). Mas, será que essa melhora é estatisticamente significativa?

# Escolhendo o teste de hipótese adequado ao problema

Como estamos lidando com 2 amostras e proporção, utilizaremos o teste de comparação de proporções.

In [ ]:
from statsmodels.stats.proportion import proportions_ztest, proportion_confint
# proportions_ztest - aplicará o teste
# proportion_confint - utilizado para calcular o intervalo de confiança

In [ ]:
resultado_controle = grupo_controle['converted']
resultado_tratamento = grupo_tratamento['converted']

# Criando uma lista com os sucessos
sucessos = [resultado_controle.sum(), resultado_tratamento.sum()]

# total de usuários
n_controle = grupo_controle.user_id.count()
n_tratamento = grupo_tratamento.user_id.count()
nobs = [n_controle, n_tratamento]

# Z_test
proportions_ztest(sucessos, nobs) #valor z e o p-valor

(-6.733481162342496, 1.6565111652777357e-11)

Nosso teste foi bilateral com 95% de confiança e 5% de significância, resultando em 2,5% em cada cauda. A zona de rejeição é z < -1.96 e z > 1.96. O valor de z obtido foi de -6.73, rejeitando assim a hipótese nula de que as proporções são iguais.

Há também a possibilidade de avaliar o valor-p, que, neste caso, foi muito inferior a 5%, rejeitando a hipótese nula de que as proporções são iguais.




# Intervalo de confiança entre as amostras.

In [ ]:
(inferior_controle, inferior_tratamento), (superior_controle, superior_tratamento) = proportion_confint(sucessos, nobs=nobs, alpha=0.05)

print(f'intervalo de confiança 95% para o grupo controle: [{inferior_controle:.3f}, {superior_controle:.3f}]')
print(f'intervalo de confiança 95% para o grupo tratamento: [{inferior_tratamento:.3f}, {superior_tratamento:.3f}]')

intervalo de confiança 95% para o grupo controle: [0.112, 0.137]
intervalo de confiança 95% para o grupo tratamento: [0.175, 0.204]


Existe um distanciamento significativo entre os intervalos de confiança das proporções nos dois grupos (não há overlap entre eles). Isso sugere que, com 95% de confiança, as proporções populacionais nos grupos controle e tratamento são estatisticamente diferentes.